# Virtual Fields Method: Student Exercise

## 🎯 Learning Objectives
In this exercise, you will:
1. Understand the VFM methodology for material identification
2. Implement noise addition to strain fields
3. Complete key calculation sections
4. Analyze the effect of noise on material parameter identification

## ⏱️ Time: 45 minutes

## 📋 Prerequisites
- CSV files: `scalarsFE.csv` and `FEM2VFM.csv`
- Understanding of strain tensors and material stiffness

---

## Part 1: Setup and Data Loading (Provided)

The following code is provided to set up the class structure and load data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import ScalarFormatter

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [2]:
class VFMPiecewiseNoise:
    """
    Virtual Fields Method implementation with noise simulation
    
    Students will complete key sections of this implementation.
    """
    
    def __init__(self, data_source='csv'):
        self.data_source = data_source
        self.data = {}
        self.results = {}
        self.noise_results = {}
    
    def load_data_from_csv(self, csv_dir='.'):
        """Load data from CSV files (PROVIDED)"""
        # Load scalar parameters
        scalar_path = os.path.join(csv_dir, 'scalarsFE.csv')
        
        if os.path.exists(scalar_path):
            with open(scalar_path, 'r') as f:
                for line in f:
                    if '=' in line:
                        key, value = line.strip().split('=')
                        key = key.strip()
                        value = float(value.strip())
                        
                        if key == 'Length':
                            self.data['L'] = value
                        elif key == 'Width':
                            self.data['w'] = value
                        elif key == 'Thick':
                            self.data['t'] = value
                        elif key == 'P':
                            self.data['F'] = value
        
        # Load FEM data
        fem_path = os.path.join(csv_dir, 'FEM2VFM.csv')
        
        if os.path.exists(fem_path):
            fem_data = pd.read_csv(fem_path, sep=r'\s+')
            
            self.data['X1'] = fem_data['X_Coord'].values
            self.data['X2'] = fem_data['Y_Coord'].values
            self.data['Eps1'] = fem_data['Eps_X'].values
            self.data['Eps2'] = fem_data['Eps_Y'].values
            self.data['Eps6'] = fem_data['Eps_XY'].values
            
            # Grid dimensions
            unique_x1 = np.unique(self.data['X1'])
            unique_x2 = np.unique(self.data['X2'])
            n_column = len(unique_x1)
            n_row = len(unique_x2)
            
            self.data['n_row'] = n_row
            self.data['n_column'] = n_column
            
            # Reshape to 2D
            try:
                self.data['X1_2D'] = self.data['X1'].reshape(n_row, n_column)
                self.data['X2_2D'] = self.data['X2'].reshape(n_row, n_column)
                self.data['Eps1_2D'] = self.data['Eps1'].reshape(n_row, n_column)
                self.data['Eps2_2D'] = self.data['Eps2'].reshape(n_row, n_column)
                self.data['Eps6_2D'] = self.data['Eps6'].reshape(n_row, n_column)
            except ValueError:
                print(f"Warning: Could not reshape to {n_row}x{n_column} grid")
        
        print("✓ Data loaded successfully")
        self._display_data_summary()
    
    def _display_data_summary(self):
        """Display loaded data summary"""
        print("\nData Summary:")
        print(f"  Specimen Length (L): {self.data.get('L', 'N/A')} mm")
        print(f"  Specimen Width (w): {self.data.get('w', 'N/A')} mm")
        print(f"  Thickness (t): {self.data.get('t', 'N/A')} mm")
        print(f"  Applied Force (F): {self.data.get('F', 'N/A')} N")
        print(f"  Grid: {self.data.get('n_row', 'N/A')} × {self.data.get('n_column', 'N/A')} points")
        print(f"  Total data points: {len(self.data.get('Eps1', []))}")

## Part 2: VFM Core Function (Provided)

This is the heart of the VFM algorithm. It's complex, so we provide it complete.
Take time to understand the structure, but you don't need to modify it.

In [3]:
    def vfm_piecewise_function(self, m=4, n=5, Eps1=None, Eps2=None, Eps6=None):
        """
        VFM piecewise function - PROVIDED COMPLETE
        
        Parameters:
        -----------
        m : int - Number of elements along e1 (x1 direction)
        n : int - Number of elements along e2 (x2 direction)
        Eps1, Eps2, Eps6 : array - Strain arrays (optional, for noise analysis)
        
        Returns:
        --------
        Q : array - Material parameters [Q11, Q22, Q12, Q66]
        eta : array - Sensitivity parameters
        Ya, Yb, Yc, Yd : array - Coefficient vectors
        """
        
        # Use provided or original data
        if Eps1 is None:
            Eps1 = self.data['Eps1']
        if Eps2 is None:
            Eps2 = self.data['Eps2']
        if Eps6 is None:
            Eps6 = self.data['Eps6']
        
        X1 = self.data['X1']
        X2 = self.data['X2']
        L = self.data['L']
        w = self.data['w']
        t = self.data['t']
        F = self.data['F']
        
        # Parameter definition
        n_nodes = (m + 1) * (n + 1)
        n_elem = m * n
        n_points = len(Eps1)
        
        n_row = self.data.get('n_row', int(np.sqrt(n_points)))
        n_column = self.data.get('n_column', int(np.sqrt(n_points)))
        
        L_el = L / m
        w_el = w / n
        
        # Adjust coordinates
        X2_adjusted = X2 - np.min(X2) + w / n_row / 2
        
        X1_vec = X1
        X2_vec = X2_adjusted
        Eps1_vec = Eps1
        Eps2_vec = Eps2
        Eps6_vec = Eps6
        
        # Element indices
        iii = np.floor(X1_vec * m / L) + 1
        jjj = np.floor(X2_vec * n / w) + 1
        
        iii = np.clip(iii, 1, m).astype(int)
        jjj = np.clip(jjj, 1, n).astype(int)
        
        xsi1 = 2 * X1_vec / L_el - iii * 2 + 1
        xsi2 = 2 * X2_vec / w_el - jjj * 2 + 1
        
        # Virtual strain calculations
        Eps1elem = np.zeros((n_points, 8))
        Eps2elem = np.zeros((n_points, 8))
        Eps6elem = np.zeros((n_points, 8))
        u1elem = np.zeros((n_points, 8))
        u2elem = np.zeros((n_points, 8))
        
        for k in range(n_points):
            Eps1elem[k, :] = np.array([
                -(1 - xsi2[k]) / 2 / L_el, 0, (1 - xsi2[k]) / 2 / L_el, 0,
                (1 + xsi2[k]) / 2 / L_el, 0, -(1 + xsi2[k]) / 2 / L_el, 0
            ])
            
            Eps2elem[k, :] = np.array([
                0, -(1 - xsi1[k]) / 2 / w_el, 0, -(1 + xsi1[k]) / 2 / w_el,
                0, (1 + xsi1[k]) / 2 / w_el, 0, (1 - xsi1[k]) / 2 / w_el
            ])
            
            Eps6elem[k, :] = np.array([
                -(1 - xsi1[k]) / w_el / 2, -(1 - xsi2[k]) / L_el / 2,
                -(1 + xsi1[k]) / w_el / 2, (1 - xsi2[k]) / L_el / 2,
                (1 + xsi1[k]) / w_el / 2, (1 + xsi2[k]) / L_el / 2,
                (1 - xsi1[k]) / w_el / 2, -(1 + xsi2[k]) / L_el / 2
            ])
        
        # Matrix assembly
        B11 = np.zeros((1, 2 * n_nodes))
        B22 = np.zeros((1, 2 * n_nodes))
        B12 = np.zeros((1, 2 * n_nodes))
        B66 = np.zeros((1, 2 * n_nodes))
        
        H11 = np.zeros((2 * n_nodes, 2 * n_nodes))
        H22 = np.zeros((2 * n_nodes, 2 * n_nodes))
        H12 = np.zeros((2 * n_nodes, 2 * n_nodes))
        H66 = np.zeros((2 * n_nodes, 2 * n_nodes))
        
        n1 = (iii - 1) * (n + 1) + jjj
        n2 = iii * (n + 1) + jjj
        n3 = iii * (n + 1) + jjj + 1
        n4 = (iii - 1) * (n + 1) + jjj + 1
        
        assemble = np.column_stack([
            n1 * 2 - 1, n1 * 2, n2 * 2 - 1, n2 * 2,
            n3 * 2 - 1, n3 * 2, n4 * 2 - 1, n4 * 2
        ]).astype(int) - 1
        
        assemble = np.clip(assemble, 0, 2 * n_nodes - 1)
        
        # Assembly loop
        for k in range(n_points):
            assemble1 = assemble[k, :]
            
            B11[0, assemble1] += Eps1_vec[k] * Eps1elem[k, :] * L * w / n_points
            B22[0, assemble1] += Eps2_vec[k] * Eps2elem[k, :] * L * w / n_points
            B12[0, assemble1] += (Eps1_vec[k] * Eps2elem[k, :] + 
                                  Eps2_vec[k] * Eps1elem[k, :]) * L * w / n_points
            B66[0, assemble1] += Eps6_vec[k] * Eps6elem[k, :] * L * w / n_points
            
            H11[np.ix_(assemble1, assemble1)] += np.outer(Eps1elem[k, :], Eps1elem[k, :])
            H22[np.ix_(assemble1, assemble1)] += np.outer(Eps2elem[k, :], Eps2elem[k, :])
            H12[np.ix_(assemble1, assemble1)] += np.outer(Eps1elem[k, :], Eps2elem[k, :])
            H66[np.ix_(assemble1, assemble1)] += np.outer(Eps6elem[k, :], Eps6elem[k, :])
        
        # Boundary conditions
        Aconst = np.zeros((4 * n + 3, 2 * n_nodes))
        
        for i in range(2 * (n + 1)):
            Aconst[i, i] = 1
        
        for i in range(n + 1):
            Aconst[i + 2 * (n + 1), 2 * n_nodes - 2 * (n + 1) + 2 * i] = 1
        
        for i in range(n):
            Aconst[i + 3 * (n + 1), 2 * n_nodes - 2 * (n + 1) + 2 * i + 1] = 1
            Aconst[i + 3 * (n + 1), 2 * n_nodes - 2 * (n + 1) + 2 * (i + 1) + 1] = -1
        
        # Z vectors
        Za = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
        Zb = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
        Zc = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
        Zd = np.zeros(2 * n_nodes + Aconst.shape[0] + 4)
        
        Za[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [1, 0, 0, 0]
        Zb[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [0, 1, 0, 0]
        Zc[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [0, 0, 1, 0]
        Zd[2 * n_nodes + Aconst.shape[0]:2 * n_nodes + Aconst.shape[0] + 4] = [0, 0, 0, 1]
        
        # Constraint matrix
        A = np.vstack([Aconst, B11, B22, B12, B66])
        B_zeros = np.zeros((A.shape[0], A.shape[0]))
        
        # Iterative optimization
        Q = np.array([1.0, 1.0, 1.0, 1.0])
        n_iter = 20
        delta_lim = 0.001
        delta = 10.0
        iteration = 1
        Q_old = Q.copy()
        
        while iteration < n_iter and delta > delta_lim:
            H = (L * w / n_points) ** 2 * (
                (Q[0] ** 2 + Q[2] ** 2) * H11 +
                (Q[1] ** 2 + Q[2] ** 2) * H22 +
                Q[3] ** 2 * H66 +
                2 * (Q[0] + Q[1]) * Q[2] * H12
            )
            
            corr = np.max(A) / np.max(H) if np.max(H) > 0 else 1.0
            
            OptM_top = np.hstack([H / 2 * corr, A.T * corr])
            OptM_bottom = np.hstack([A, B_zeros])
            OptM = np.vstack([OptM_top, OptM_bottom])
            
            try:
                Ya = np.linalg.solve(OptM, Za)
                Yb = np.linalg.solve(OptM, Zb)
                Yc = np.linalg.solve(OptM, Zc)
                Yd = np.linalg.solve(OptM, Zd)
                
                Ya = Ya[:2 * n_nodes]
                Yb = Yb[:2 * n_nodes]
                Yc = Yc[:2 * n_nodes]
                Yd = Yd[:2 * n_nodes]
                
                Q[0] = Ya[2 * n_nodes - 1] * F / t
                Q[1] = Yb[2 * n_nodes - 1] * F / t
                Q[2] = Yc[2 * n_nodes - 1] * F / t
                Q[3] = Yd[2 * n_nodes - 1] * F / t
                
                delta = np.sum((Q_old - Q) ** 2 / Q ** 2)
                
                iteration += 1
                Q_old = Q.copy()
                
            except np.linalg.LinAlgError as e:
                print(f"Error at iteration {iteration}: {e}")
                break
        
        # Final Hessian for sensitivity
        H_final = (L * w / n_points) ** 2 * (
            (Q[0] ** 2 + Q[2] ** 2) * H11 +
            (Q[1] ** 2 + Q[2] ** 2) * H22 +
            Q[3] ** 2 * H66 +
            2 * (Q[0] + Q[1]) * Q[2] * H12
        )
        
        # Sensitivity parameters
        eta = np.zeros(4)
        try:
            eta[0] = np.sqrt(Ya.T @ H_final @ Ya)
            eta[1] = np.sqrt(Yb.T @ H_final @ Yb)
            eta[2] = np.sqrt(Yc.T @ H_final @ Yc)
            eta[3] = np.sqrt(Yd.T @ H_final @ Yd)
        except:
            eta = np.zeros(4)
        
        return Q, eta, Ya, Yb, Yc, Yd

# Add this method to the class
VFMPiecewiseNoise.vfm_piecewise_function = vfm_piecewise_function

---
## 🎓 STUDENT EXERCISE SECTION
---

### Exercise 1: Add Gaussian White Noise (10 minutes)

**Theory**: In real experiments, strain measurements contain noise. We simulate this by adding Gaussian white noise:

$$\varepsilon_{\text{noisy}} = \varepsilon_{\text{clean}} + \mathcal{N}(0, \sigma^2)$$

Where $\mathcal{N}(0, \sigma^2)$ is a normal distribution with mean 0 and standard deviation $\sigma$.

**Task**: Complete the `add_noise_to_strains()` function below.

In [4]:
def add_noise_to_strains(self, noise_amplitude):
    """
    Add Gaussian white noise to strain fields
    
    Parameters:
    -----------
    noise_amplitude : float
        Standard deviation of the Gaussian noise (e.g., 10e-4)
    
    Returns:
    --------
    Eps1_noisy, Eps2_noisy, Eps6_noisy : arrays with added noise
    """
    
    # Get original clean strain data
    Eps1_orig = self.data['Eps1']
    Eps2_orig = self.data['Eps2']
    Eps6_orig = self.data['Eps6']
    
    # TODO: Generate Gaussian noise for each strain component
    # Hint: Use np.random.randn() with the same shape as the strain arrays
    # Multiply by noise_amplitude to set the standard deviation
    
    noise1 = # YOUR CODE HERE
    noise2 = # YOUR CODE HERE  
    noise6 = # YOUR CODE HERE
    
    # TODO: Add noise to original strains
    Eps1_noisy = # YOUR CODE HERE
    Eps2_noisy = # YOUR CODE HERE
    Eps6_noisy = # YOUR CODE HERE
    
    # Store for later use
    self.Eps1_noisy = Eps1_noisy
    self.Eps2_noisy = Eps2_noisy
    self.Eps6_noisy = Eps6_noisy
    
    return Eps1_noisy, Eps2_noisy, Eps6_noisy

# Add method to class
VFMPiecewiseNoise.add_noise_to_strains = add_noise_to_strains

SyntaxError: invalid syntax (436940457.py, line 24)

### Exercise 2: Compare Clean vs Noisy Results (10 minutes)

**Theory**: The VFM algorithm identifies material parameters Q = [Q11, Q22, Q12, Q66].
We want to assess how noise affects these identifications.

**Task**: Complete the `run_noise_comparison()` function.

In [ ]:
def run_noise_comparison(self, m=4, n=5, noise_amplitude=10e-4):
    """
    Compare VFM results with and without noise
    
    Parameters:
    -----------
    m, n : int - Mesh elements in x and y directions
    noise_amplitude : float - Noise standard deviation
    """
    
    print(f"\n{'='*70}")
    print("NOISE COMPARISON ANALYSIS")
    print(f"{'='*70}")
    print(f"Mesh: {m}×{n} elements")
    print(f"Noise amplitude (σ): {noise_amplitude:.1e}\n")
    
    # TODO: Run VFM with clean data
    # Hint: Call self.vfm_piecewise_function(m, n) with no strain arguments
    print("Running VFM with clean data...")
    Q_clean, eta_clean, _, _, _, _ = # YOUR CODE HERE
    
    # TODO: Add noise to strains
    # Hint: Call self.add_noise_to_strains(noise_amplitude)
    print("Adding noise to strain fields...")
    Eps1_noisy, Eps2_noisy, Eps6_noisy = # YOUR CODE HERE
    
    # TODO: Run VFM with noisy data
    # Hint: Call self.vfm_piecewise_function with noisy strain arguments
    print("Running VFM with noisy data...")
    Q_noisy, eta_noisy, _, _, _, _ = # YOUR CODE HERE
    
    # Convert to GPa
    Q_clean_gpa = Q_clean / 1e3
    Q_noisy_gpa = Q_noisy / 1e3
    
    # Display results
    self._display_results(Q_clean_gpa, Q_noisy_gpa)
    
    # Store results
    self.results = {
        'Q_clean': Q_clean,
        'Q_noisy': Q_noisy,
        'eta_clean': eta_clean,
        'eta_noisy': eta_noisy
    }
    
    return Q_clean_gpa, Q_noisy_gpa

def _display_results(self, Q_clean, Q_noisy):
    """Display comparison results"""
    params = ['Q11', 'Q22', 'Q12', 'Q66']
    
    print(f"\n{'='*70}")
    print("RESULTS")
    print(f"{'='*70}")
    print(f"{'Parameter':<10} {'Clean (GPa)':<15} {'Noisy (GPa)':<15} {'Diff %':<10}")
    print("-"*70)
    
    for i, param in enumerate(params):
        diff_pct = abs(Q_clean[i] - Q_noisy[i]) / Q_clean[i] * 100
        print(f"{param:<10} {Q_clean[i]:<15.4f} {Q_noisy[i]:<15.4f} {diff_pct:<10.3f}")

# Add methods to class
VFMPiecewiseNoise.run_noise_comparison = run_noise_comparison
VFMPiecewiseNoise._display_results = _display_results

### Exercise 3: Monte Carlo Analysis (15 minutes)

**Theory**: To quantify noise effects statistically, we run multiple iterations with different noise realizations:

- **Mean**: Average identified value
- **Standard deviation (σ)**: Spread of results
- **Coefficient of variation (CV)**: $CV = \frac{\sigma}{\mu} \times 100\%$

**Task**: Complete the Monte Carlo loop.

In [ ]:
def run_monte_carlo(self, m=4, n=5, noise_amplitude=10e-4, n_iterations=20):
    """
    Run Monte Carlo simulation with multiple noise realizations
    
    Parameters:
    -----------
    m, n : int - Mesh density
    noise_amplitude : float - Noise standard deviation
    n_iterations : int - Number of Monte Carlo iterations
    """
    
    print(f"\n{'='*70}")
    print("MONTE CARLO NOISE ANALYSIS")
    print(f"{'='*70}")
    print(f"Iterations: {n_iterations}")
    print(f"Mesh: {m}×{n} elements")
    print(f"Noise amplitude: {noise_amplitude:.1e}\n")
    
    # TODO: Initialize empty lists to store results from each iteration
    Q11_results = # YOUR CODE HERE (empty list)
    Q22_results = # YOUR CODE HERE
    Q12_results = # YOUR CODE HERE
    Q66_results = # YOUR CODE HERE
    
    print(f"Running {n_iterations} iterations...")
    
    # TODO: Loop through iterations
    for i in range(n_iterations):
        print(f"  Iteration {i+1}/{n_iterations}", end="")
        
        # TODO: Add noise and run VFM
        # Hint: Use self.add_noise_to_strains() and self.vfm_piecewise_function()
        Eps1_n, Eps2_n, Eps6_n = # YOUR CODE HERE
        Q, _, _, _, _, _ = # YOUR CODE HERE
        
        # TODO: Append results (convert to GPa by dividing by 1e3)
        Q11_results.append(# YOUR CODE HERE)
        Q22_results.append(# YOUR CODE HERE)
        Q12_results.append(# YOUR CODE HERE)
        Q66_results.append(# YOUR CODE HERE)
        
        print(" ✓")
    
    # Convert to numpy arrays
    self.noise_results = {
        'Q11': np.array(Q11_results),
        'Q22': np.array(Q22_results),
        'Q12': np.array(Q12_results),
        'Q66': np.array(Q66_results)
    }
    
    # Display statistics
    self._display_monte_carlo_stats()
    
    return self.noise_results

def _display_monte_carlo_stats(self):
    """Display Monte Carlo statistics"""
    print(f"\n{'='*70}")
    print("STATISTICAL RESULTS")
    print(f"{'='*70}")
    print(f"{'Param':<8} {'Mean (GPa)':<15} {'Std (GPa)':<15} {'CV %':<10}")
    print("-"*70)
    
    params = ['Q11', 'Q22', 'Q12', 'Q66']
    
    for param in params:
        values = self.noise_results[param]
        # TODO: Calculate statistics
        mean_val = # YOUR CODE HERE (use np.mean)
        std_val = # YOUR CODE HERE (use np.std)
        cv_val = # YOUR CODE HERE (coefficient of variation in %)
        
        print(f"{param:<8} {mean_val:<15.4f} {std_val:<15.5f} {cv_val:<10.3f}")

# Add methods to class
VFMPiecewiseNoise.run_monte_carlo = run_monte_carlo
VFMPiecewiseNoise._display_monte_carlo_stats = _display_monte_carlo_stats

### Exercise 4: Plotting Results (10 minutes)

**Task**: Complete the histogram plotting function for Monte Carlo results.

In [ ]:
def plot_monte_carlo_histograms(self):
    """
    Plot histograms of Monte Carlo results
    """
    if not self.noise_results:
        print("No Monte Carlo results available. Run Monte Carlo analysis first.")
        return
    
    params = ['Q11', 'Q22', 'Q12', 'Q66']
    
    # TODO: Create 2x2 subplot figure
    fig, axes = # YOUR CODE HERE (use plt.subplots)
    axes = axes.ravel()
    
    for i, param in enumerate(params):
        values = self.noise_results[param]
        
        # TODO: Calculate mean
        mean_val = # YOUR CODE HERE
        
        # TODO: Create histogram
        # Hint: Use axes[i].hist() with appropriate parameters
        axes[i].hist(# YOUR CODE HERE - add values, bins, color, etc.)
        
        # TODO: Add vertical line for mean
        axes[i].axvline(# YOUR CODE HERE)
        
        # Labels
        axes[i].set_xlabel(f'{param} (GPa)', fontsize=12)
        axes[i].set_ylabel('Frequency', fontsize=12)
        axes[i].set_title(f'{param} Distribution', fontsize=14, fontweight='bold')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('monte_carlo_results.png', dpi=150, bbox_inches='tight')
    print("\nHistograms saved as 'monte_carlo_results.png'")
    plt.show()

# Add method to class
VFMPiecewiseNoise.plot_monte_carlo_histograms = plot_monte_carlo_histograms

---
## 🚀 Running Your Code
---

### Step 1: Load Data

In [ ]:
# Initialize and load data
vfm = VFMPiecewiseNoise(data_source='csv')
vfm.load_data_from_csv('.')  # Make sure CSV files are in current directory

### Step 2: Run Noise Comparison

In [ ]:
# Single noise comparison
Q_clean, Q_noisy = vfm.run_noise_comparison(
    m=5,  # horizontal elements
    n=4,  # vertical elements
    noise_amplitude=10e-4
)

### Step 3: Run Monte Carlo Analysis

In [ ]:
# Monte Carlo analysis
results = vfm.run_monte_carlo(
    m=5,
    n=4,
    noise_amplitude=10e-4,
    n_iterations=20  # Keep low for speed (20-50 recommended)
)

### Step 4: Plot Results

In [ ]:
# Plot Monte Carlo histograms
vfm.plot_monte_carlo_histograms()

---
## 📊 Discussion Questions

After running your analysis, consider:

1. **Noise Impact**: Which parameter (Q11, Q22, Q12, Q66) is most sensitive to noise? Why?

2. **Coefficient of Variation**: What does the CV tell us about the reliability of each parameter identification?

3. **Mesh Dependency**: Try different mesh configurations (m×n). How does mesh refinement affect noise sensitivity?

4. **Physical Interpretation**: 
   - Q11: Longitudinal stiffness
   - Q22: Transverse stiffness  
   - Q12: Coupling stiffness
   - Q66: Shear stiffness
   
   Which makes sense to be most/least sensitive?

---

## 🎓 Bonus Exercises (If Time Permits)

1. **Vary Noise Amplitude**: Test noise_amplitude = [5e-4, 10e-4, 20e-4]. Plot CV vs noise amplitude.

2. **Compare Reference Values**: If you have reference material properties, calculate % error for each parameter.

3. **Advanced Plotting**: Add error bars or confidence intervals to your plots.

---

## ✅ Solution Checklist

Before considering your exercise complete, verify:

- [ ] Exercise 1: Noise correctly added using np.random.randn()
- [ ] Exercise 2: Both clean and noisy VFM analyses run successfully
- [ ] Exercise 3: Monte Carlo loop stores all iterations correctly
- [ ] Exercise 3: Statistics (mean, std, CV) calculated properly
- [ ] Exercise 4: Histograms display with mean lines
- [ ] All code runs without errors
- [ ] Results make physical sense (positive stiffness values)

---

**Good luck! 🎯**